### Simple Approach

In [1]:
s = "very long corpus..."
words = s.split(" ")  
vocabulary = dict(enumerate(set(words)))  

In [2]:
vocabulary

{0: 'corpus...', 1: 'very', 2: 'long'}

Tokenizers librarary was designed so that it provides all the required blocks to create end-to-end tokenizers in an interchangeable way. In that sense, it provides these various components:

- Normalizer: Executes all the initial transformations over the initial input string. For example when you need to lowercase some text, maybe strip it, or even apply one of the common unicode normalization process, you will add a Normalizer.
- PreTokenizer: In charge of splitting the initial input string. That's the component that decides where and how to pre-segment the origin string. The simplest example would be like we saw above, to simply split on spaces.
- Model: Handles all the sub-token discovery and generation, this part is trainable and really dependant on input data.
- Post-Processor: Provides advanced construction features to be compatible with some of the Transformers-based SoTA models. For instance, for BERT it would wrap the tokenized sentence around [CLS] and [SEP] tokens.
- Decoder: In charge of mapping back a tokenized input to the original string. The decoder is usually chosen according to the PreTokenizer we used previously.
- Trainer: Provides training capabilities to each model.

For each of the components above tokenizers library provides multiple implementations:

- Normalizer: Lowercase, Unicode (NFD, NFKD, NFC, NFKC), Bert, Strip, ...
- PreTokenizer: ByteLevel, WhitespaceSplit, CharDelimiterSplit, Metaspace, ...
- Model: WordLevel, BPE, WordPiece
- Post-Processor: BertProcessor, ...
- Decoder: WordLevel, BPE, WordPiece, ...

All of these building blocks can be combined to create working tokenization pipelines.

### Subtoken Tokenization

In [3]:
BIG_FILE_URL = 'https://raw.githubusercontent.com/dscape/spell/master/test/resources/big.txt'

from requests import get
with open('big.txt', 'wb') as big_f:
    response = get(BIG_FILE_URL, )
    
    if response.status_code == 200:
        big_f.write(response.content)
    else:
        print("Unable to get the file: {}".format(response.reason))

In [4]:
# Everything described below can be replaced by the ByteLevelBPETokenizer class. 

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.trainers import BpeTrainer

In [5]:
# Instantiate empty tokenizer
tokenizer = Tokenizer(BPE())

In [6]:
# Now we enable lower-casing and unicode-normalization
# The Sequence normalizer allows us to combine multiple Normalizer that will be executed in order.

tokenizer.normalizer = Sequence([
    NFKC(),
    Lowercase()
])

In [7]:
# Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
tokenizer.pre_tokenizer = ByteLevel()

# And finally, let's plug a decoder so we can recover from a tokenized input to the original one
tokenizer.decoder = ByteLevelDecoder()

In [8]:
# We initialize our trainer, giving it the details about the vocabulary we want to generate
trainer = BpeTrainer(vocab_size = 25000, show_progress = True, initial_alphabet = ByteLevel.alphabet())
tokenizer.train(trainer, ["big.txt"])

print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 25000


In [9]:
tokenizer.model.save('tokenizers/')

['tokenizers/vocab.json', 'tokenizers/merges.txt']

In [10]:
tokenizer.model = BPE('tokenizers/vocab.json', 'tokenizers/merges.txt')
encoding = tokenizer.encode("This is a simple input to be tokenized")

print("Encoded string: {}".format(encoding.tokens))

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded string: ['Ġthis', 'Ġis', 'Ġa', 'Ġsimple', 'Ġin', 'put', 'Ġto', 'Ġbe', 'Ġtoken', 'ized']
Decoded string:  this is a simple input to be tokenized


#### The Encoding structure exposes multiple properties which are useful when working with transformers models

- normalized_str: The input string after normalization (lower-casing, unicode, stripping, etc.)
- original_str: The input string as it was provided
- tokens: The generated tokens with their string representation
- input_ids: The generated tokens with their integer representation
- tention_mask: If your input has been padded by the tokenizer, then this would be a vector of 1 for any non padded token and 0 for padded ones.
- special_token_mask: If your input contains special tokens such as [CLS], [SEP], [MASK], [PAD], then this would be a vector with 1 in places where a special token has been added.
- type_ids: If your input was made of multiple "parts" such as (question, context), then this would be a vector with for each token the segment it belongs to.
- overflowing: If your input has been truncated into multiple subparts because of a length limit (for BERT for example the sequence length is limited to 512), this will contain all the remaining overflowing parts.